In [4]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
import os

%matplotlib inline

In [5]:
!ls /Users/ChrisD/Documents/UCBerkeleyMIDS/W210/Capstone_Project/Data

BR_categories_grouped.csv           US_BATON_ROUGE_trainingdata 3.tsv
BR_comments_cut.csv                 US_CHICAGO_trainingdata.tsv
CHI_categories_grouped.csv          US_CHI_BR_IE_trainingdata.tsv
CIN_categories_grouped.csv          chi_br_IE_train_1.tsv
Complaint_types - Master_sheet.tsv  chicago_comments_cut.csv
Complaint_types_Master_sheet_v2.tsv ireland_fix_my_street.csv
US_BATON_ROUGE_trainingdata 2.tsv


In [6]:
PATH='~/Documents/UCBerkeleyMIDS/W210/Capstone_Project/Data'
#os.chdir(path)

# Create dictionary for reclassifying categories

"Complaint_types - Master_sheet.tsv" is a table of each category-subcategory used in the Chicago, Baton Rouge, Cinncinnati, and other files along with their corresponding MAIN and SUB category that we will be using for training.

In [73]:
df_reclass=pd.read_csv(PATH+'/Complaint_types_Master_sheet_v2.tsv',sep='\t')

#replace and drop columns with no CATEGORY_MAIN
df_reclass['CATEGORY_MAIN'].replace('', np.nan, inplace=True)
df_reclass.dropna(subset=['CATEGORY_MAIN'], inplace=True)

df_reclass['CATEGORY_MAIN']=df_reclass['CATEGORY_MAIN'].map(lambda x:x.upper())
df_reclass['CATEGORY_SUB']=df_reclass['CATEGORY_SUB'].map(lambda x:x.upper())

Fields:
    - CITY : which city's 311 program
    - DEPT_311: second-most specific categorization for each complaint
    - CODE_311: most specific categorization for each complaint 
    - COUNT: How many complaints of each CODE_311
    - CATEGORY_MAIN: our general categorization
    - CATEGORY_SUB: our specific categorization
    
We are only training on two levels of categorization. The initial plan is focus on the higher level categorization, so that complaints can get to the correct department. Subcategorization can either be done as a second classification process within each category or done all at once in the main categorization process.

In [30]:
df_reclass.head(2)

,CITY,DEPT_311,CODE_311,COUNT,CATEGORY_MAIN,CATEGORY_SUB
0,US_CHICAGO,health_environment,abandoned site,87.0,ENVIRONMENT,ENVIRONMENT_ABANDONED_SITE
1,US_CHICAGO,health_environment,air pollution work order,17839.0,ENVIRONMENT,ENVIRONMENT_AIR_POLLUTION


The counts below are unedited, precleaned data

In [31]:
display(df_reclass.groupby(['CATEGORY_MAIN']).sum())
df_reclass.groupby(['CATEGORY_MAIN','CATEGORY_SUB']).sum()

,COUNT
CATEGORY_MAIN,
ENVIRONMENT,256382.0
FIRE,7222.0
GENERAL,0.0
GOVERNANCE,2805.0
HOUSING,57622.0
INFRASTRUCTURE,32185.0
PLANNING,23403.0
PUBLIC HEALTH,8823.0
PUBLIC ORDER,8815.0


COUNT
CATEGORY_MAIN  CATEGORY_SUB                                 
ENVIRONMENT    ENVIRONEMNT_DEAD_ANIMAL                 270.0
               ENVIRONMENT_ABANDONED_SITE             3405.0
               ENVIRONMENT_ABANDONED_VEHICLE          2823.0
               ENVIRONMENT_AIR_POLLUTION             17905.0
               ENVIRONMENT_ASBESTOS                   2737.0
               ENVIRONMENT_DEAD_ANIMAL                 720.0
               ENVIRONMENT_DUMPING                   16927.0
               ENVIRONMENT_GARBAGE_COLLECTION       116772.0
               ENVIRONMENT_GENERAL                    3608.0
               ENVIRONMENT_HAZARDOUS_MATERIAL         5827.0
               ENVIRONMENT_LITTER                    15658.0
               ENVIRONMENT_OVERGROWTH                41630.0
               ENVIRONMENT_RECYCLING                 28051.0
               ENVIRONMENT_WATER_POLLUTION              34.0
               ENVIRONNMENT_HAZARDOUS_MATERIAL          15.0
FIRE           FIRE_EQUIPMENT_BROKEN                    11.0
               FIRE_GENERAL                           7211.0
GENERAL        GENERAL_EMERGENCY                         0.0
               GENERAL_INQUIRY                           0.0
GOVERNANCE     GOVERNANCE_GENERAL                       32.0
               GOVERNANCE_SIGNAGE                     2773.0
HOUSING        HOUSING_FLOODING                          0.0
               HOUSING_GENERAL                       42523.0
               HOUSING_HEALTH_CODE                    5645.0
               HOUSING_MOLD                           5462.0
               HOUSING_PESTS                           164.0
               HOUSING_SAFETY                         3828.0
INFRASTRUCTURE INFRASTRUCTURE_GENERAL                27626.0
               INFRASTRUCTURE_WATER_REPAIR            3814.0
               INFRASTRUCTURE_WATER_UNSAFE             745.0
PLANNING       PLANNING_CONSTRUCTION                   202.0
               PLANNING_GENERAL                      18768.0
               PLANNING_UNSAFE_ENVIRONMENT            4433.0
PUBLIC HEALTH  PUBLICHEALTH_ANIMAL                       9.0
               PUBLICHEALTH_ANIMAL_FECES               711.0
               PUBLICHEALTH_GENERAL                   2191.0
               PUBLICHEALTH_PESTS                     5854.0
               PUBLICHEALTH_PUBLIC_BUILDING_HYGENE      18.0
               PUBLICHEALTH_SCHOOL_HYGENE               40.0
PUBLIC ORDER   PUBLICORDER_GENERAL                      77.0
               PUBLICORDER_NOISE_COMPLAINT            8738.0
STREET         STREET_CLEANING                        1661.0
               STREET_GENERAL                         2764.0
               STREET_LIGHTING                        7166.0
               STREET_REPAIR                         20257.0
               STREET_SEWAR                           4706.0
               STREET_URGENT_REPAIR                   1276.0
VANDALISM      VANDALISM_GENERAL                       429.0
               VANDALISM_GRAFFITI                      718.0

# Support functions

In [32]:
def lookup_311_code(dic,city,dept_311,code_311):
    
    # dic= dictionary of {(city,dept_311,code_311): (MAIN,SUB)}
    
    vals=dic.get((city,dept_311,code_311))
    
    if vals is not None:
        return vals
    else:
        return ('unknown','unknown')


## Create the dictionary

We use the df_reclass dataframe to build a dictionary of DEPT_311 and CODE_311 conversions for each city 

In [74]:
reclass_dict=defaultdict(dict)


for index,row in df_reclass.iterrows():

    reclass_dict[(row['CITY'],row['DEPT_311'],row['CODE_311'])]=(row['CATEGORY_MAIN'],row['CATEGORY_SUB'])

In [75]:
reclass_dict

defaultdict(dict,
            {('IRELAND', 'GENERAL', 'Broken Street Lights'): ('STREET',
              'STREET_LIGHTING'),
             ('IRELAND', 'GENERAL', 'Broken Street Paths'): ('STREET',
              'STREET_REPAIR'),
             ('IRELAND', 'GENERAL', 'Broken street lighting'): ('STREET',
              'STREET_LIGHTING'),
             ('IRELAND', 'GENERAL', 'Broken street path'): ('STREET',
              'STREET_REPAIR'),
             ('IRELAND', 'GENERAL', 'Drainage Problems'): ('INFRASTRUCTURE',
              'INFRASTRUCTURE_GENERAL'),
             ('IRELAND', 'GENERAL', 'Graffiti'): ('VANDALISM',
              'VANDALISM_GRAFFITI'),
             ('IRELAND', 'GENERAL', 'Leaks and Drainage'): ('INFRASTRUCTURE',
              'INFRASTRUCTURE_GENERAL'),
             ('IRELAND',
              'GENERAL',
              'Litter and Illegal Dumping'): ('ENVIRONMENT', 'ENVIRONMENT_DUMPING'),
             ('IRELAND', 'GENERAL', 'Road Maintenance/Potholes'): ('STREET',
              

# DATA CLEAN AND JOIN

We can either import the full dataset and trim it down, or import the already trimmed dataframe 

In [43]:
!ls /Users/ChrisD/Documents/UCBerkeleyMIDS/W210/Capstone_Project/Data

BR_categories_grouped.csv           Complaint_types - Master_sheet.tsv
BR_comments_cut.csv                 Complaint_types_Master_sheet_v2.tsv
CHI_categories_grouped.csv          chicago_comments_cut.csv
CIN_categories_grouped.csv          ireland_fix_my_street.csv


## Chicago

In [102]:
df_chicago=pd.read_csv(PATH+'/chicago_comments_cut.csv', sep='\t')
df_chicago.drop(labels=['Unnamed: 0','INSPECTOR','locality'],axis=1,inplace=True)
df_chicago.head(2)

,COMPLAINT ID,DATA SOURCE,COMPLAINT TYPE,COMPLAINT DATE,COMPLAINT DETAIL,INSPECTION LOG
0,1725,HISTORIC DEPT. OF ENVIRONMENT,permits issued by doe work order,10/03/2011,QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...,[INSPECTION LOG #: 1723 03-OCT-11 18:55:00] TH...
1,600301602,DEPT. OF PUBLIC HEALTH,illegal dumping work order,07/14/2017,CONSTRUCTION/DEMOLITION DEBRIS DUMPED IN BACK ...,[INSPECTION LOG #: 3439804 14-JUL-17 09:45:00]...


In [103]:
df_chicago['DATA SOURCE']='health_environment'
df_chicago['CITY']='US_CHICAGO'
df_chicago['COMPLAINT ID']= df_chicago['CITY'] + '_' + df_chicago['COMPLAINT ID']
df_chicago.columns=['COMPLAINT_ID','DEPT_311','CODE_311','COMPLAINT DATE','COMPLAINT_1','COMPLAINT_2','CITY']
df_chicago=df_chicago[['COMPLAINT_ID','CITY','DEPT_311','CODE_311','COMPLAINT_1','COMPLAINT_2','COMPLAINT DATE']]
df_chicago.head(2)

,COMPLAINT_ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE
0,US_CHICAGO_1725,US_CHICAGO,health_environment,permits issued by doe work order,QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...,[INSPECTION LOG #: 1723 03-OCT-11 18:55:00] TH...,10/03/2011
1,US_CHICAGO_600301602,US_CHICAGO,health_environment,illegal dumping work order,CONSTRUCTION/DEMOLITION DEBRIS DUMPED IN BACK ...,[INSPECTION LOG #: 3439804 14-JUL-17 09:45:00]...,07/14/2017


In [104]:
df_chicago['CATEGORY_MAIN']=df_chicago.apply(lambda x: lookup_311_code(reclass_dict,x.CITY,x.DEPT_311,x.CODE_311)[0],axis=1)



In [105]:
df_chicago['CATEGORY_SUB']=df_chicago.apply(lambda x: lookup_311_code(reclass_dict,x.CITY,x.DEPT_311,x.CODE_311)[1],axis=1)



In [106]:
df_chicago.to_csv(PATH+'/US_CHICAGO_trainingdata.tsv',sep='\t')


## Baton Rouge

In [107]:
df_batonrouge=pd.read_csv(PATH+'/BR_comments_cut.csv')
display(df_batonrouge.head(2))

df_batonrouge.drop(labels=['Unnamed: 0','DEPARTMENT','DIVISION','GEOLOCATION'],axis=1,inplace=True)
df_batonrouge['CITY']='US_BATON_ROUGE'
df_batonrouge.columns=['COMPLAINT_ID','COMPLAINT DATE','DEPT_311','CODE_311','COMPLAINT_1','CITY']
df_batonrouge['COMPLAINT_ID']=df_batonrouge['CITY']+ '_' + df_batonrouge['COMPLAINT_ID'].map(str)
df_batonrouge['COMPLAINT_2']=None
df_batonrouge=df_batonrouge[['COMPLAINT_ID','CITY','DEPT_311','CODE_311','COMPLAINT_1','COMPLAINT_2','COMPLAINT DATE']]
df_batonrouge.head(2)

,Unnamed: 0,SERVICE REQUEST ID,CREATE DATE,PARENT TYPE,TYPE,DEPARTMENT,DIVISION,COMMENTS,GEOLOCATION
0,0,220943,03/22/2016 04:40:00 PM,ROAD MAINTENANCE ISSUES,POTHOLE,MAINTENANCE,STREET MAINTENANCE,POTHOLE ON THE SW CORNER OF HILMONT AND SPRING...,"(30.342981, -91.114136)"
1,1,248677,07/27/2016 02:45:00 PM,STREET/TRAFFIC ISSUES,ROAD STRIPING NEEDED,TRANSPORTATION AND DEVELOPMENT,TRAFFIC ENGINEERING,STREET NEEDS TO BE RE STRIPED AT THE CORNER SO...,"(30.439724, -91.174988)"


,COMPLAINT_ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE
0,US_BATON_ROUGE_220943,US_BATON_ROUGE,ROAD MAINTENANCE ISSUES,POTHOLE,POTHOLE ON THE SW CORNER OF HILMONT AND SPRING...,None,03/22/2016 04:40:00 PM
1,US_BATON_ROUGE_248677,US_BATON_ROUGE,STREET/TRAFFIC ISSUES,ROAD STRIPING NEEDED,STREET NEEDS TO BE RE STRIPED AT THE CORNER SO...,None,07/27/2016 02:45:00 PM


In [108]:
df_batonrouge['CATEGORY_MAIN']=df_batonrouge.apply(lambda x: lookup_311_code(reclass_dict,x.CITY,x.DEPT_311,x.CODE_311)[0],axis=1)


In [109]:
df_batonrouge['CATEGORY_SUB']=df_batonrouge.apply(lambda x: lookup_311_code(reclass_dict,x.CITY,x.DEPT_311,x.CODE_311)[1],axis=1)

df_batonrouge.head(2)

,COMPLAINT_ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE,CATEGORY_MAIN,CATEGORY_SUB
0,US_BATON_ROUGE_220943,US_BATON_ROUGE,ROAD MAINTENANCE ISSUES,POTHOLE,POTHOLE ON THE SW CORNER OF HILMONT AND SPRING...,None,03/22/2016 04:40:00 PM,STREET,STREET_REPAIR
1,US_BATON_ROUGE_248677,US_BATON_ROUGE,STREET/TRAFFIC ISSUES,ROAD STRIPING NEEDED,STREET NEEDS TO BE RE STRIPED AT THE CORNER SO...,None,07/27/2016 02:45:00 PM,STREET,STREET_REPAIR


In [110]:
df_batonrouge.to_csv(PATH+'/US_BATON_ROUGE_trainingdata.tsv',sep='\t')

# IRELAND

In [111]:
df_IE=pd.read_csv(PATH+'/ireland_fix_my_street.csv')
df_IE.drop(labels=['Unnamed: 0'],axis=1,inplace=True)
display(df_IE.head(2))
df_IE['COMPLAINT DATE']='01/01/2018'
df_IE['CITY']='IRELAND'
df_IE['DEPT_311']='GENERAL'
df_IE=df_IE[['COMPLAINT_ID','CITY','DEPT_311','CODE_311','COMPLAINT_1','COMPLAINT_2','COMPLAINT DATE']]

,COMPLAINT_ID,CODE_311,COMPLAINT_1,COMPLAINT_2
0,ireland_100,Road or path defects,Potholes in Allenwood South,"Hi, I live in Allenwood South (L70044), my hou..."
1,ireland_10000,Street Lighting,Street Light,Street light not working


In [112]:
df_IE.head()

,COMPLAINT_ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE
0,ireland_100,IRELAND,GENERAL,Road or path defects,Potholes in Allenwood South,"Hi, I live in Allenwood South (L70044), my hou...",01/01/2018
1,ireland_10000,IRELAND,GENERAL,Street Lighting,Street Light,Street light not working,01/01/2018
2,ireland_10001,IRELAND,GENERAL,Tree and Grass Maintenance,Ditch removal,Ditch trees and brambles recently removed by t...,01/01/2018
3,ireland_10002,IRELAND,GENERAL,Street Lighting,Street Light,Street light not working,01/01/2018
4,ireland_10003,IRELAND,GENERAL,Street Lighting,Street Light,Street light not working at this location,01/01/2018


In [113]:
df_IE['CATEGORY_MAIN']=df_IE.apply(lambda x: lookup_311_code(reclass_dict,x.CITY,x.DEPT_311,x.CODE_311)[0],axis=1)

In [114]:
df_IE['CATEGORY_SUB']=df_IE.apply(lambda x: lookup_311_code(reclass_dict,x.CITY,x.DEPT_311,x.CODE_311)[1],axis=1)

df_IE.head(2)

,COMPLAINT_ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE,CATEGORY_MAIN,CATEGORY_SUB
0,ireland_100,IRELAND,GENERAL,Road or path defects,Potholes in Allenwood South,"Hi, I live in Allenwood South (L70044), my hou...",01/01/2018,STREET,STREET_REPAIR
1,ireland_10000,IRELAND,GENERAL,Street Lighting,Street Light,Street light not working,01/01/2018,STREET,STREET_LIGHTING


## COMBINE THEM

In [115]:
display(df_chicago.columns)==display(df_batonrouge.columns)
display(df_IE.columns)==display(df_batonrouge.columns)

Index(['COMPLAINT_ID', 'CITY', 'DEPT_311', 'CODE_311', 'COMPLAINT_1',
       'COMPLAINT_2', 'COMPLAINT DATE', 'CATEGORY_MAIN', 'CATEGORY_SUB'],
      dtype='object')

Index(['COMPLAINT_ID', 'CITY', 'DEPT_311', 'CODE_311', 'COMPLAINT_1',
       'COMPLAINT_2', 'COMPLAINT DATE', 'CATEGORY_MAIN', 'CATEGORY_SUB'],
      dtype='object')

Index(['COMPLAINT_ID', 'CITY', 'DEPT_311', 'CODE_311', 'COMPLAINT_1',
       'COMPLAINT_2', 'COMPLAINT DATE', 'CATEGORY_MAIN', 'CATEGORY_SUB'],
      dtype='object')

Index(['COMPLAINT_ID', 'CITY', 'DEPT_311', 'CODE_311', 'COMPLAINT_1',
       'COMPLAINT_2', 'COMPLAINT DATE', 'CATEGORY_MAIN', 'CATEGORY_SUB'],
      dtype='object')

True

In [116]:
df_combined=pd.concat([df_chicago,df_batonrouge,df_IE])

In [117]:
display(df_chicago.shape)
display(df_batonrouge.shape)
display(df_IE.shape)
df_combined.shape

(48579, 9)

(232537, 9)

(23090, 9)

(304206, 9)

In [118]:
df_combined.head()

,COMPLAINT_ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE,CATEGORY_MAIN,CATEGORY_SUB
0,US_CHICAGO_1725,US_CHICAGO,health_environment,permits issued by doe work order,QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...,[INSPECTION LOG #: 1723 03-OCT-11 18:55:00] TH...,10/03/2011,ENVIRONMENT,ENVIRONMENT_GENERAL
1,US_CHICAGO_600301602,US_CHICAGO,health_environment,illegal dumping work order,CONSTRUCTION/DEMOLITION DEBRIS DUMPED IN BACK ...,[INSPECTION LOG #: 3439804 14-JUL-17 09:45:00]...,07/14/2017,ENVIRONMENT,ENVIRONMENT_DUMPING
2,US_CHICAGO_1106,US_CHICAGO,health_environment,air pollution work order,REPLACING WINDOWS AND SANDBLASTING CAUSING DUS...,[INSPECTION LOG #: 1104 19-AUG-11 11:06:00] MY...,08/19/2011,ENVIRONMENT,ENVIRONMENT_AIR_POLLUTION
3,US_CHICAGO_2063,US_CHICAGO,health_environment,air pollution work order,NaN,[INSPECTION LOG #: 2167 16-OCT-11 13:08:00] NA...,10/26/2011,ENVIRONMENT,ENVIRONMENT_AIR_POLLUTION
4,US_CHICAGO_2503,US_CHICAGO,health_environment,abandoned site,PROPERTY CAUSING HAZARDOUS CONDITIONS ON PUBLI...,[INSPECTION LOG #: 2451 14-DEC-11 23:50:00] IN...,12/14/2011,ENVIRONMENT,ENVIRONMENT_ABANDONED_SITE


In [119]:
df_combined.to_csv(PATH+'/US_CHI_BR_IE_trainingdata.tsv',sep='\t')

In [120]:
display(df_combined.head(2))
display(df_combined.tail(2))

,COMPLAINT_ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE,CATEGORY_MAIN,CATEGORY_SUB
0,US_CHICAGO_1725,US_CHICAGO,health_environment,permits issued by doe work order,QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...,[INSPECTION LOG #: 1723 03-OCT-11 18:55:00] TH...,10/03/2011,ENVIRONMENT,ENVIRONMENT_GENERAL
1,US_CHICAGO_600301602,US_CHICAGO,health_environment,illegal dumping work order,CONSTRUCTION/DEMOLITION DEBRIS DUMPED IN BACK ...,[INSPECTION LOG #: 3439804 14-JUL-17 09:45:00]...,07/14/2017,ENVIRONMENT,ENVIRONMENT_DUMPING


,COMPLAINT_ID,CITY,DEPT_311,CODE_311,COMPLAINT_1,COMPLAINT_2,COMPLAINT DATE,CATEGORY_MAIN,CATEGORY_SUB
23088,ireland_9998,IRELAND,GENERAL,Street Lighting,Lighting problem,Street light not working,01/01/2018,STREET,STREET_LIGHTING
23089,ireland_9999,IRELAND,GENERAL,Street Lighting,Street Light,Street light not working,01/01/2018,STREET,STREET_LIGHTING


# TRAINING DATA

In [121]:
df_combined['CITY'].unique()

array(['US_CHICAGO', 'US_BATON_ROUGE', 'IRELAND'], dtype=object)

In [134]:
#TRAINING 1

#Chicago use COMPLAINT 1
display(df_combined[df_combined['CITY']=='US_CHICAGO'][['COMPLAINT_ID','CATEGORY_MAIN','CATEGORY_SUB','COMPLAINT_1']].head())
chi_train_1=df_combined[df_combined['CITY']=='US_CHICAGO'][['COMPLAINT_ID','CATEGORY_MAIN','CATEGORY_SUB','COMPLAINT_1']]
chi_train_1.columns=['COMPLAINT_ID','CATEGORY_MAIN','CATEGORY_SUB','COMPLAINT']

# BATON ROUGE use COMPLAINT_1
display(df_combined[df_combined['CITY']=='US_BATON_ROUGE'][['COMPLAINT_ID','CATEGORY_MAIN','CATEGORY_SUB','COMPLAINT_1']].head())
br_train_1=df_combined[df_combined['CITY']=='US_BATON_ROUGE'][['COMPLAINT_ID','CATEGORY_MAIN','CATEGORY_SUB','COMPLAINT_1']]
br_train_1.columns=['COMPLAINT_ID','CATEGORY_MAIN','CATEGORY_SUB','COMPLAINT']

# IRELAND use COMPLAINT_2
display(df_combined[df_combined['CITY']=='IRELAND'][['COMPLAINT_ID','CATEGORY_MAIN','CATEGORY_SUB','COMPLAINT_2']].head())
IE_train_1=df_combined[df_combined['CITY']=='IRELAND'][['COMPLAINT_ID','CATEGORY_MAIN','CATEGORY_SUB','COMPLAINT_2']]
IE_train_1.columns=['COMPLAINT_ID','CATEGORY_MAIN','CATEGORY_SUB','COMPLAINT']


chi_br_IE_train_1=pd.concat([chi_train_1,br_train_1,IE_train_1])
display(chi_br_IE_train_1.shape)
display(chi_br_IE_train_1.head())
chi_br_IE_train_1.dropna(subset=['COMPLAINT'], inplace=True)
print('na complaints dropped')
display(chi_br_IE_train_1.shape)
chi_br_IE_train_1.to_csv(PATH+'/chi_br_IE_train_1.tsv',sep='\t')


,COMPLAINT_ID,CATEGORY_MAIN,CATEGORY_SUB,COMPLAINT_1
0,US_CHICAGO_1725,ENVIRONMENT,ENVIRONMENT_GENERAL,QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...
1,US_CHICAGO_600301602,ENVIRONMENT,ENVIRONMENT_DUMPING,CONSTRUCTION/DEMOLITION DEBRIS DUMPED IN BACK ...
2,US_CHICAGO_1106,ENVIRONMENT,ENVIRONMENT_AIR_POLLUTION,REPLACING WINDOWS AND SANDBLASTING CAUSING DUS...
3,US_CHICAGO_2063,ENVIRONMENT,ENVIRONMENT_AIR_POLLUTION,NaN
4,US_CHICAGO_2503,ENVIRONMENT,ENVIRONMENT_ABANDONED_SITE,PROPERTY CAUSING HAZARDOUS CONDITIONS ON PUBLI...


,COMPLAINT_ID,CATEGORY_MAIN,CATEGORY_SUB,COMPLAINT_1
0,US_BATON_ROUGE_220943,STREET,STREET_REPAIR,POTHOLE ON THE SW CORNER OF HILMONT AND SPRING...
1,US_BATON_ROUGE_248677,STREET,STREET_REPAIR,STREET NEEDS TO BE RE STRIPED AT THE CORNER SO...
2,US_BATON_ROUGE_251335,INFRASTRUCTURE,INFRASTRUCTURE_GENERAL,"STORM DRAINS N FRONTOF HOMR ARE CAVING IN, LI..."
3,US_BATON_ROUGE_286906,HOUSING,HOUSING_GENERAL,VACANT PROPERTY AT 2230 MARYLAND STREET IS UNS...
4,US_BATON_ROUGE_293409,STREET,STREET_REPAIR,ROAD IS SINKING ROAD SUB BASE NEED TO BE REPAI...


,COMPLAINT_ID,CATEGORY_MAIN,CATEGORY_SUB,COMPLAINT_2
0,ireland_100,STREET,STREET_REPAIR,"Hi, I live in Allenwood South (L70044), my hou..."
1,ireland_10000,STREET,STREET_LIGHTING,Street light not working
2,ireland_10001,ENVIRONMENT,ENVIRONMENT_OVERGROWTH,Ditch trees and brambles recently removed by t...
3,ireland_10002,STREET,STREET_LIGHTING,Street light not working
4,ireland_10003,STREET,STREET_LIGHTING,Street light not working at this location


(304206, 4)

,COMPLAINT_ID,CATEGORY_MAIN,CATEGORY_SUB,COMPLAINT
0,US_CHICAGO_1725,ENVIRONMENT,ENVIRONMENT_GENERAL,QUESTIONABLE BUSINESS PRACTICES REGARDING OILS...
1,US_CHICAGO_600301602,ENVIRONMENT,ENVIRONMENT_DUMPING,CONSTRUCTION/DEMOLITION DEBRIS DUMPED IN BACK ...
2,US_CHICAGO_1106,ENVIRONMENT,ENVIRONMENT_AIR_POLLUTION,REPLACING WINDOWS AND SANDBLASTING CAUSING DUS...
3,US_CHICAGO_2063,ENVIRONMENT,ENVIRONMENT_AIR_POLLUTION,NaN
4,US_CHICAGO_2503,ENVIRONMENT,ENVIRONMENT_ABANDONED_SITE,PROPERTY CAUSING HAZARDOUS CONDITIONS ON PUBLI...


na complaints dropped


(298362, 4)

In [9]:
chi_br_IE_train_1=pd.read_csv(PATH+'/chi_br_IE_train_1.tsv',sep='\t')
chi_br_IE_train_1.drop(labels=['Unnamed: 0'],axis=1,inplace=True)

In [10]:
chi_br_IE_train_1.groupby(['CATEGORY_MAIN']).count().sort_values('COMPLAINT_ID', ascending=False)

,COMPLAINT_ID,CATEGORY_SUB,COMPLAINT
CATEGORY_MAIN,,,
ENVIRONMENT,214901,214901,214901
STREET,37770,37770,37770
INFRASTRUCTURE,26793,26793,26793
PUBLIC ORDER,8644,8644,8644
HOUSING,5279,5279,5279
GOVERNANCE,2466,2466,2466
PLANNING,1365,1365,1365
VANDALISM,1144,1144,1144


In [11]:
#chi_br_IE_train_1.groupby(['CATEGORY_MAIN','CATEGORY_SUB']).count().sort_values('COMPLAINT_ID', ascending=False)


chi_br_IE_train_1.groupby(['CATEGORY_MAIN']).apply(lambda x: (x.groupby('CATEGORY_SUB').count().sort_values('COMPLAINT_ID', ascending=False)))


COMPLAINT_ID  CATEGORY_MAIN  \
CATEGORY_MAIN  CATEGORY_SUB                                                   
ENVIRONMENT    ENVIRONMENT_GARBAGE_COLLECTION         114890         114890   
               ENVIRONMENT_RECYCLING                   27129          27129   
               ENVIRONMENT_OVERGROWTH                  19138          19138   
               ENVIRONMENT_AIR_POLLUTION               17743          17743   
               ENVIRONMENT_DUMPING                     14332          14332   
               ENVIRONMENT_HAZARDOUS_MATERIAL           5757           5757   
               ENVIRONMENT_GENERAL                      3555           3555   
               ENVIRONMENT_ABANDONED_SITE               3392           3392   
               ENVIRONMENT_ABANDONED_VEHICLE            2814           2814   
               ENVIRONMENT_ASBESTOS                     2711           2711   
               ENVIRONMENT_LITTER                       2465           2465   
               ENVIRONMENT_DEAD_ANIMAL                   656            656   
               ENVIRONEMNT_DEAD_ANIMAL                   270            270   
               ENVIRONMENT_WATER_POLLUTION                34             34   
               ENVIRONNMENT_HAZARDOUS_MATERIAL            15             15   
GOVERNANCE     GOVERNANCE_SIGNAGE                       2466           2466   
HOUSING        HOUSING_HEALTH_CODE                      4587           4587   
               HOUSING_GENERAL                           415            415   
               HOUSING_SAFETY                            277            277   
INFRASTRUCTURE INFRASTRUCTURE_GENERAL                  25194          25194   
               INFRASTRUCTURE_WATER_REPAIR               929            929   
               INFRASTRUCTURE_WATER_UNSAFE               670            670   
PLANNING       PLANNING_GENERAL                          999            999   
               PLANNING_CONSTRUCTION                     195            195   
               PLANNING_UNSAFE_ENVIRONMENT               171            171   
PUBLIC ORDER   PUBLICORDER_NOISE_COMPLAINT              8644           8644   
STREET         STREET_REPAIR                           20217          20217   
               STREET_LIGHTING                          7166           7166   
               STREET_SEWAR                             4703           4703   
               STREET_GENERAL                           2756           2756   
               STREET_CLEANING                          1654           1654   
               STREET_URGENT_REPAIR                     1274           1274   
VANDALISM      VANDALISM_GRAFFITI                        718            718   
               VANDALISM_GENERAL                         426            426   

                                                COMPLAINT  
CATEGORY_MAIN  CATEGORY_SUB                                
ENVIRONMENT    ENVIRONMENT_GARBAGE_COLLECTION      114890  
               ENVIRONMENT_RECYCLING                27129  
               ENVIRONMENT_OVERGROWTH               19138  
               ENVIRONMENT_AIR_POLLUTION            17743  
               ENVIRONMENT_DUMPING                  14332  
               ENVIRONMENT_HAZARDOUS_MATERIAL        5757  
               ENVIRONMENT_GENERAL                   3555  
               ENVIRONMENT_ABANDONED_SITE            3392  
               ENVIRONMENT_ABANDONED_VEHICLE         2814  
               ENVIRONMENT_ASBESTOS                  2711  
               ENVIRONMENT_LITTER                    2465  
               ENVIRONMENT_DEAD_ANIMAL                656  
               ENVIRONEMNT_DEAD_ANIMAL                270  
               ENVIRONMENT_WATER_POLLUTION             34  
               ENVIRONNMENT_HAZARDOUS_MATERIAL         15  
GOVERNANCE     GOVERNANCE_SIGNAGE                    2466  
HOUSING        HOUSING_HEALTH_CODE                   4587  
               HOUSING_GENERAL                        415  
   

# APPENDIX OF SCRATCH WORK

In [105]:

reclass_dict=defaultdict(list)


for index,row in df_reclass.iterrows():
    
    # add new city or append to city
    # add or append Department class with 311 sub category
    # get the new MAIN and SUB classes
    
    reclass_dict[row['CITY']].append({\
        row['DEPT_311']:{row['CODE_311']:\
                         {'CATEGORY_MAIN': row['CATEGORY_MAIN'],'CATEGORY_SUB':row['CATEGORY_SUB']}}})  

    
reclass_dict 

defaultdict(list,
            {'US_BATON_ROUGE': [{'BLIGHTED PROPERTIES': {'DIRTY OR UNFENCED SWIMMING POOL': {'CATEGORY_MAIN': 'HOUSING',
                 'CATEGORY_SUB': 'HOUSING_SAFETY'}}},
              {'BLIGHTED PROPERTIES': {'ILLEGAL SIGN IN THE PUBLIC RIGHT-OF-WAY': {'CATEGORY_MAIN': 'GOVERNANCE',
                 'CATEGORY_SUB': 'GOVERNANCE_SIGNAGE'}}},
              {'BLIGHTED PROPERTIES': {'JUNK, INOPERABLE, OR ABANDONED VEHICLE': {'CATEGORY_MAIN': 'ENVIRONMENT',
                 'CATEGORY_SUB': 'ENVIRONMENT_ABANDONED_VEHICLE'}}},
              {'BLIGHTED PROPERTIES': {'JUNK, TRASH, OR DEBRIS ON PRIVATE PROPERTY': {'CATEGORY_MAIN': 'HOUSING',
                 'CATEGORY_SUB': 'HOUSING_HEALTH_CODE'}}},
              {'BLIGHTED PROPERTIES': {'PROPERTY MISSING WINDOWS OR DOORS': {'CATEGORY_MAIN': 'HOUSING',
                 'CATEGORY_SUB': 'HOUSING_GENERAL'}}},
              {'BLIGHTED PROPERTIES': {'PROPERTY NEEDS TO BE CONDEMNED OR TORN DOWN': {'CATEGORY_MAIN': 'ENVIRONMENT',


In [142]:
reclass_dict=defaultdict(dict)


for index,row in df_reclass.iterrows():
    
    # add new city or append to city
    # add or append Department class with 311 sub category
    # get the new MAIN and SUB classes
    dummy_dict={'CATEGORY_MAIN': row['CATEGORY_MAIN'],'CATEGORY_SUB':row['CATEGORY_SUB']}
    reclass_dict.update({row['CITY']:{row['DEPT_311']:{row['CODE_311']:dummy_dict}}})  

    
reclass_dict 

defaultdict(dict,
            {'US_BATON_ROUGE': {'STREET/TRAFFIC ISSUES': {'SCHOOL ZONE/SCHOOL FLASHER ISSUE': {'CATEGORY_MAIN': 'GOVERNANCE',
                'CATEGORY_SUB': 'GOVERNANCE_SIGNAGE'}}},
             'US_CHICAGO': {'health_environment': {'other': {'CATEGORY_MAIN': 'ENVIRONMENT',
                'CATEGORY_SUB': 'ENVIRONMENT_GENERAL'}}}})

In [146]:
df_reclass.groupby(['CITY','DEPT_311','CODE_311']).apply(lambda x: [dict(zip(x.CATEGORY_MAIN, x.CATEGORY_SUB))]).to_dict()

{('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'DIRTY OR UNFENCED SWIMMING POOL'): [{'HOUSING': 'HOUSING_SAFETY'}],
 ('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'ILLEGAL SIGN IN THE PUBLIC RIGHT-OF-WAY'): [{'GOVERNANCE': 'GOVERNANCE_SIGNAGE'}],
 ('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'JUNK, INOPERABLE, OR ABANDONED VEHICLE'): [{'ENVIRONMENT': 'ENVIRONMENT_ABANDONED_VEHICLE'}],
 ('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'JUNK, TRASH, OR DEBRIS ON PRIVATE PROPERTY'): [{'HOUSING': 'HOUSING_HEALTH_CODE'}],
 ('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'PROPERTY MISSING WINDOWS OR DOORS'): [{'HOUSING': 'HOUSING_GENERAL'}],
 ('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'PROPERTY NEEDS TO BE CONDEMNED OR TORN DOWN'): [{'ENVIRONMENT': 'ENVIRONMENT_ABANDONED_SITE'}],
 ('US_BATON_ROUGE',
  'BLIGHTED PROPERTIES',
  'TALL GRASS'): [{'ENVIRONMENT': 'ENVIRONMENT_OVERGROWTH'}],
 ('US_BATON_ROUGE',
  'CODE VIOLATIONS',
  'BUILDING CODE VIOLATIONS'): [{'PLANNING': 'PLANNING_UNSAFE_ENVIRO

In [177]:
dummy=defaultdict(dict)

dummy[('US_CHICAGO','health_environment','other')]=('a','b')
dummy[('US_CHICAGO','health_environment','service stations/storage tanks work order')]=('a','c')

display(dummy)
display(dummy[('US_CHICAGO','health_environment','other')])
display(dummy.get(('US_CHICAGO','health_environment','service stations/storage tanks work order'))[1])
#display(dummy.get(('US_CHICAGO','health_environment','recycling work order'))[1])

defaultdict(dict,
            {('US_CHICAGO', 'health_environment', 'other'): ('a', 'b'),
             ('US_CHICAGO',
              'health_environment',
              'service stations/storage tanks work order'): ('a', 'c')})

('a', 'b')

'c'

In [185]:
def lookup_311_code(dic,city,dept_311,code_311):
    
    vals=dic.get((city,dept_311,code_311))
    
    if vals is not None:
        return vals
    else:
        return ('unknown','unknown')
    


lookup_311_code(dummy,'US_CHICAGO','health_environment','other')[0]

'a'

In [115]:

reclass_dictlist=list()


for index,row in df_reclass.iterrows():
    
    # add new city or append to city
    # add or append Department class with 311 sub category
    # get the new MAIN and SUB classes
    
    reclass_dictlist.append({row['CITY']:\
        {row['DEPT_311']:{row['CODE_311']:\
        {'CATEGORY_MAIN': row['CATEGORY_MAIN'],'CATEGORY_SUB':row['CATEGORY_SUB']}}}})  

    
reclass_dict 

defaultdict(list,
            {'US_BATON_ROUGE': [{'BLIGHTED PROPERTIES': {'DIRTY OR UNFENCED SWIMMING POOL': {'CATEGORY_MAIN': 'HOUSING',
                 'CATEGORY_SUB': 'HOUSING_SAFETY'}}},
              {'BLIGHTED PROPERTIES': {'ILLEGAL SIGN IN THE PUBLIC RIGHT-OF-WAY': {'CATEGORY_MAIN': 'GOVERNANCE',
                 'CATEGORY_SUB': 'GOVERNANCE_SIGNAGE'}}},
              {'BLIGHTED PROPERTIES': {'JUNK, INOPERABLE, OR ABANDONED VEHICLE': {'CATEGORY_MAIN': 'ENVIRONMENT',
                 'CATEGORY_SUB': 'ENVIRONMENT_ABANDONED_VEHICLE'}}},
              {'BLIGHTED PROPERTIES': {'JUNK, TRASH, OR DEBRIS ON PRIVATE PROPERTY': {'CATEGORY_MAIN': 'HOUSING',
                 'CATEGORY_SUB': 'HOUSING_HEALTH_CODE'}}},
              {'BLIGHTED PROPERTIES': {'PROPERTY MISSING WINDOWS OR DOORS': {'CATEGORY_MAIN': 'HOUSING',
                 'CATEGORY_SUB': 'HOUSING_GENERAL'}}},
              {'BLIGHTED PROPERTIES': {'PROPERTY NEEDS TO BE CONDEMNED OR TORN DOWN': {'CATEGORY_MAIN': 'ENVIRONMENT',


In [118]:
reclass_dictlist[0]
reclass_dictlist.index(['US_CHICAGO']['health_environment']['abandoned site'])

TypeError: list indices must be integers or slices, not str

In [102]:

reclass_dict=dict()


for index,row in df_reclass.iterrows():
    
    # add new city or append to city
    # add or append Department class with 311 sub category
    # get the new MAIN and SUB classes
    
    reclass_dict=dict(reclass_dict,**{row['CITY']:{
        row['DEPT_311']:{row['CODE_311']:\
                         {'CATEGORY_MAIN': row['CATEGORY_MAIN'],'CATEGORY_SUB':row['CATEGORY_SUB']}}}}  )

    
reclass_dict 

{'US_BATON_ROUGE': {'STREET/TRAFFIC ISSUES': {'SCHOOL ZONE/SCHOOL FLASHER ISSUE': {'CATEGORY_MAIN': 'GOVERNANCE',
    'CATEGORY_SUB': 'GOVERNANCE_SIGNAGE'}}},
 'US_CHICAGO': {'health_environment': {'other': {'CATEGORY_MAIN': 'ENVIRONMENT',
    'CATEGORY_SUB': 'ENVIRONMENT_GENERAL'}}}}

In [90]:
from collections import Counter
reclass_dict=dict()


for index,row in df_reclass.iterrows():
    
    # add new city or append to city
    # add or append Department class with 311 sub category
    # get the new MAIN and SUB classes
    dummydict={'CATEGORY_MAIN': row['CATEGORY_MAIN'],'CATEGORY_SUB':row['CATEGORY_SUB']}
    reclass_dict+={row['CITY']:{row['DEPT_311']:{row['CODE_311']}}}
    

    
reclass_dict 

TypeError: unsupported operand type(s) for +=: 'dict' and 'dict'